In [6]:
import re
import csv
import requests
from PyPDF2 import PdfReader
from io import BytesIO

def download_pdf(url):
    # Sends a GET request to the URL
    response = requests.get(url)
    # Raise an error for bad responses
    response.raise_for_status()
    # Return the content of the response
    return BytesIO(response.content)

def extract_station_data(pdf_file):
    # Open the PDF file from a BytesIO stream
    reader = PdfReader(pdf_file)
    full_text = ""
    
    # Extract text from each page of the PDF
    for page in reader.pages:
        full_text += page.extract_text() + "\n"

    # Regular expression to find relevant station details
    pattern = re.compile(r"(\b[A-Z]{2}\b) ([\w\s]+)\n[\w\s,()]+[\n]([\d\.]+) N ([\d\.]+) W")

    # Extract all data matching the pattern
    return re.findall(pattern, full_text)

def save_to_csv(data, filename):
    # Field names for the CSV
    fields = ['State', 'Station Name', 'Latitude', 'Longitude']

    # Writing to csv file
    with open(filename, 'w', newline='') as csvfile:
        # Creating a csv writer object
        csvwriter = csv.writer(csvfile)
        
        # Writing the fields
        csvwriter.writerow(fields)
        
        # Writing the data rows
        csvwriter.writerows(data)

def main():
    pdf_url = "https://www.ncei.noaa.gov/pub/data/uscrn/documentation/site/photos/stationsbystate_lores.pdf"
    output_csv = "Weather_Stations.csv"
    
    # Download the PDF from the URL
    pdf_file = download_pdf(pdf_url)
    
    # Extract data from PDF
    station_data = extract_station_data(pdf_file)
    print (len(station_data))
    # Save data to CSV
    save_to_csv(station_data, output_csv)
    
    print(f"Data extracted and saved to {output_csv}")

if __name__ == "__main__":
    main()


104
Data extracted and saved to Weather_Stations.csv


In [8]:
from staticmap import StaticMap, CircleMarker

m = StaticMap(800, 600, url_template='http://a.tile.osm.org/{z}/{x}/{y}.png')

# Example: Add a marker at the coordinates for central London
# The coordinates need to be longitude, latitude
marker = CircleMarker((0, 51.5074), 'red', 9)  # Adjust the coordinates as needed
m.add_marker(marker)

# Render and save the map image
image = m.render()
image.save('my_map.png')


In [9]:
import pandas as pd
from staticmap import StaticMap, CircleMarker
from PIL import Image, ImageSequence

def generate_map_image(longitude, latitude, output_path):
    """Generate a static map image for a given location."""
    m = StaticMap(800, 600, url_template='http://a.tile.osm.org/{z}/{x}/{y}.png')
    marker = CircleMarker((longitude, latitude), 'red', 9)
    m.add_marker(marker)
    image = m.render()
    image.save(output_path)

def create_gif(images, output_path, duration=300):
    """Create an animated GIF from a list of image files."""
    frames = [Image.open(image) for image in images]
    frames[0].save(output_path, save_all=True, append_images=frames[1:], optimize=False, duration=duration, loop=0)

def main():
    # Read locations from CSV
    df = pd.read_csv('Weather_Stations.csv')

    image_paths = []
    for index, row in df.iterrows():
        output_path = f"map_{index}.png"
        generate_map_image(float(row['Longitude']), float(row['Latitude']), output_path)
        image_paths.append(output_path)

    # Create an animated GIF
    create_gif(image_paths, 'weather_stations.gif')

if __name__ == "__main__":
    main()
